### 朴素贝叶斯算法实现

In [2]:
import pandas as pd


def create_data():
    # 生成示例数据
    data = {"x": ['r', 'g', 'r', 'b', 'g', 'g', 'r', 'r', 'b', 'g', 'g', 'r', 'b', 'b', 'g'],
            "y": ['m', 's', 'l', 's', 'm', 's', 'm', 's', 'm', 'l', 'l', 's', 'm', 'm', 'l'],
            "labels": ['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'B']}
    data = pd.DataFrame(data, columns=["labels", "x", "y"])
    return data
data = create_data()
data

,labels,x,y
0,A,r,m
1,A,g,s
2,A,r,l
3,A,b,s
4,A,g,m
5,A,g,s
6,A,r,m
7,A,r,s
8,B,b,m
9,B,g,l


In [3]:
def get_P_labels(labels):
    # P(\text{种类}) 先验概率计算
    labels = list(labels)  # 转换为 list 类型
    P_label = {}  # 设置空字典用于存入 label 的概率
    for label in labels:
        # 统计 label 标签在标签集中出现的次数再除以总长度
        P_label[label] = labels.count(
            label) / float(len(labels))  # p = count(y) / count(Y)
    return P_label


P_labels = get_P_labels(data["labels"])
P_labels

{'A': 0.5333333333333333, 'B': 0.4666666666666667}

In [4]:
import numpy as np

# 将 data 中的属性切割出来，即 x 和 y 属性
train_data = np.array(data.iloc[:, 1:])
train_data

array([['r', 'm'],
       ['g', 's'],
       ['r', 'l'],
       ['b', 's'],
       ['g', 'm'],
       ['g', 's'],
       ['r', 'm'],
       ['r', 's'],
       ['b', 'm'],
       ['g', 'l'],
       ['g', 'l'],
       ['r', 's'],
       ['b', 'm'],
       ['b', 'm'],
       ['g', 'l']], dtype=object)

In [7]:
labels = data["labels"]
label_index = []
# 遍历所有的标签，这里就是将标签为 A 和 B 的数据集分开，label_index 中存的是该数据的下标
for y in P_labels.keys():
    temp_index = []
    # enumerate 函数返回 Series 类型数的索引和值，其中 i 为索引，label 为值
    for i, label in enumerate(labels):
        if (label == y):
            temp_index.append(i)
        else:
            pass
    label_index.append(temp_index)
label_index

[[0, 1, 2, 3, 4, 5, 6, 7], [8, 9, 10, 11, 12, 13, 14]]

In [10]:
def get_P_fea_lab(P_label, features, data):
    # P(\text{特征}∣种类) 先验概率计算
    # 该函数就是求 种类为 P_label 条件下特征为 features 的概率
    P_fea_lab = {}
    train_data = data.iloc[:, 1:]
    train_data = np.array(train_data)
    labels = data["labels"]
    # 遍历所有的标签
    for each_label in P_label.keys():
        # 上面代码的另一种写法，这里就是将标签为 A 和 B 的数据集分开，label_index 中存的是该数据的下标
        label_index = [i for i, label in enumerate(
            labels) if label == each_label]

        # 遍历该属性下的所有取值
        # 求出每种标签下，该属性取每种值的概率
        for j in range(len(features)):
            # 筛选出该属性下属性值为 features[j] 的数据
            feature_index = [i for i, feature in enumerate(
                train_data[:, j]) if feature == features[j]]

            # set(x_index)&set(y_index) 取交集，得到标签值为 each_label,属性值为 features[j] 的数据集合
            fea_lab_count = len(set(feature_index) & set(label_index))
            key = str(features[j]) + '|' + str(each_label)  # 拼接字符串

            # 计算先验概率
            # 计算 labels 为 each_label下，featurs 为 features[j] 的概率值
            P_fea_lab[key] = fea_lab_count / float(len(label_index))
    return P_fea_lab


features = ['r', 'm']
get_P_fea_lab(P_labels, features, data)

{'r|A': 0.5,
 'm|A': 0.375,
 'r|B': 0.14285714285714285,
 'm|B': 0.42857142857142855}

In [12]:
def classify(data, features):
    # 朴素贝叶斯分类器
    # 求 labels 中每个 label 的先验概率
    labels = data['labels']
    # 这里也就是求 P（B），P_label 为一个字典，存的是每个 B 对应的 P(B)
    P_label = get_P_labels(labels)
    P_fea_lab = get_P_fea_lab(P_label, features, data)  # 这里是在求 P（A|B）

    P = {}
    P_show = {}  # 后验概率
    for each_label in P_label:
        P[each_label] = P_label[each_label]
        # 遍历每个标签下的每种属性
        for each_feature in features:
            # 拼接字符串为 P(B/A) 用于字典的键值
            key = str(each_label)+'|'+str(features)
            # 计算 P(B/A) = P(B) * P(A/B) 因为所有的后验概率，分母相同。因此，在计算时可以忽略掉。
            P_show[key] = P[each_label] * \
                P_fea_lab[str(each_feature) + '|' + str(each_label)]
            # 把刚才算的概率放到 P 列表里面，这个 P 列表的键值变成了标签。
            # 这样做的目的，其实是为了在后面取最大时，取出就是标签，而不是 标签|特征
            P[each_label] = P[each_label] * \
                P_fea_lab[str(each_feature) + '|' +
                          str(each_label)]
    # 输出 P_show 和 P 观察，发现他们的概率值没有变，只是字典的 key 值变了
    print(P_show)
    print(P)
    features_label = max(P, key=P.get)  # 概率最大值对应的类别
    return features_label

In [13]:
classify(data, ['r', 'm'])

{"A|['r', 'm']": 0.1, "B|['r', 'm']": 0.02857142857142857}
{'A': 0.1, 'B': 0.02857142857142857}


'A'

### 应用：朴素贝叶斯垃圾邮件分类

数据源地址
https://plg.uwaterloo.ca/cgi-bin/cgiwrap/gvcormac/foo06
文件：trec06c.tgz

In [16]:
data = pd.read_table('data/trec06c/full/index', header=None,
                     encoding='gb2312', delim_whitespace=True)
data.head()

,0,1
0,spam,../data/000/000
1,ham,../data/000/001
2,spam,../data/000/002
3,spam,../data/000/003
4,spam,../data/000/004


In [38]:
df = data.replace(['spam', 'ham'], [0, 1])  # 0 替代 spam，1 替代 ham
df = df.replace(regex=["\.."], value='data/trec06c')  # 替换掉文件路径
df = df.sample(len(df), random_state=1, )[:10000]  # 打乱样本并取前 1 万条数据
df.groupby(0).count()  # 统计样本

,1
0,
0,6595
1,3405


统计样本量之后，垃圾邮件有 6595 个，正常邮件有 3405 个。

数据预处理：  
1,转换源数据编码格式为 UTF-8 格式。  
2,过滤字符：去除所有非中文字符，如标点符号、英文字符、数字、网站链接等特殊字符。  
3,过滤停用词。  
4,对邮件内容进行分词处理。  

通过正则表达式滤掉了所有英文，数字，标点符号，特殊符号，只保留汉字。
同时，通过 Unicode 中文编码范围 0x4e00-0x9fff 过滤一些长相奇怪的文字。

In [19]:
import re

def clean_str(line):
    # 清理邮件，替换不需要的字符串
    line.strip('\n')
    line = re.sub(r"[^\u4e00-\u9fff]", "", line)
    line = re.sub(
        "[0-9a-zA-Z\-\s+\.\!\/_,$%^*\(\)\+(+\"\')]+|[+——！，。？、~@#￥%……&*（）<>\[\]:：★◆【】《》;；=?？]+", "", line)
    return line.strip()

In [25]:
def load_stopwords(file_path):
    # 加载停用词
    with open(file_path, 'r',encoding='utf8') as f:
        stopwords = [line.strip('\n') for line in f.readlines()]
    return stopwords

stopwords = load_stopwords('data\stopwords.txt')
stopwords

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '--',
 '.',
 '..',
 '...',
 '......',
 '...................',
 './',
 '.一',
 '.数',
 '.日',
 '/',
 '//',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '://',
 '::',
 ';',
 '<',
 '=',
 '>',
 '>>',
 '?',
 '@',
 'A',
 'Lex',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'exp',
 'sub',
 'sup',
 '|',
 '}',
 '~',
 '~~~~',
 '·',
 '×',
 '×××',
 'Δ',
 'Ψ',
 'γ',
 'μ',
 'φ',
 'φ．',
 'В',
 '—',
 '——',
 '———',
 '‘',
 '’',
 '’‘',
 '“',
 '”',
 '”，',
 '…',
 '……',
 '…………………………………………………③',
 '′∈',
 '′｜',
 '℃',
 'Ⅲ',
 '↑',
 '→',
 '∈［',
 '∪φ∈',
 '≈',
 '①',
 '②',
 '②ｃ',
 '③',
 '③］',
 '④',
 '⑤',
 '⑥',
 '⑦',
 '⑧',
 '⑨',
 '⑩',
 '──',
 '■',
 '▲',
 '\u3000',
 '、',
 '。',
 '〈',
 '〉',
 '《',
 '》',
 '》），',
 '」',
 '『',
 '』',
 '【',
 '】',
 '〔',
 '〕',
 '〕〔',
 '㈧',
 '一',
 '一.',
 '一一',
 '一下',
 '一个',
 '一些',
 '一何',
 '一切',
 '一则',
 '一则通过',
 '一天',
 '一定',
 '一方面',
 '一旦',
 '一时',
 '一来',
 '一样',
 '一次',
 '一片',
 '一番',
 '一直',
 '一致',
 '一般',
 '一起',


In [26]:
import jieba

def process(file_path, test_mode=False):
    # 清洗一封邮件
    '''
    - file_path: 文本文件路径
    - test_mode: 测试模式，后文我们会将一个字符串写入文件(utf-8 编码)，而训练文件以 GBK 编码，
                 如果自己实现分类，请注意编码格式，通常为 utf-8
    - return: words, 处理、分词之后的有效词语
    '''
    words = []
    with open(file_path, 'rb') as f:
        for line in f.readlines():
            if not test_mode:
                line = line.strip().decode("gbk", 'ignore')
            else:
                line = line.strip().decode("utf-8", 'ignore')
            line = clean_str(line)
            if len(line) == 0:
                continue
            seg_list = list(jieba.cut(line, cut_all=False))
            for x in seg_list:
                if len(x) <= 1:
                    continue
                if x in stopwords:
                    continue
                words.append(x)
    return words

In [28]:
words = process('data/trec06c/data/000/000')
" ".join(words)

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\delia\AppData\Local\Temp\jieba.cache
Loading model cost 1.110 seconds.
Prefix dict has been built succesfully.


'上海 培训 课程 财务 纠淼 沙盘 模拟 财务 课程 背景 一位 管理 技术人员 懂得 技术 角度 衡量 合算 方案 也许 却是 财务 陷阱 表面 赢利 亏损 使经 营者 接受 技术手段 财务 运作 相结合 每位 管理 技术人员 老板 角度 思考 规避 财务 陷阱 管理决策 目标 一致性 课程 沙盘 模拟 案例 分析 企业 管理 技术人员 财务管理 知识 利用 财务 信息 改进 管理决策 管理 效益 最大化 学习 课程 会计 财务管理 提高 日常 管理 活动 财务 可行性 业绩 评价 方法 评估 业绩 实施 科学 业绩考核 合乎 财务 墓芾 老板 思维 同步 分析 关键 业绩 指标 战略规划 预算 企业 管理 重心 管理 系统性 课程 大纲 财务 工作 内容 作用 财务会计 财务 专家 思维 模式 财务 工作 内容 管理者 利用 财务 管理 决策 阅读 分析 财务报表 会计报表 损益表 阅读 分析 资产 负债表 阅读 分析 资金 流量 现金流量 阅读 分析 会计报表 之间 关系 会计报表 读懂 企业 状况 案例 分析 报表 判断 企业 业绩 水平 财务 手段 成本 控制 产品成本 概念 本浚利 分析 标准 成本 制度 成本 控制 作用 目标 成本法 控制 产品成本 保证 利润 水平 作业 成本法 管理 分析 实施 精细 成本 管理 沉没 成本 机会成本 正确 决策 改善 采购 生产 环节 运作 改良 企业 整体 财务状况 综合 案例 分析 财务 尚械 墓芾 醴桨 管理 技术 方案 可行性 分析 产品开发 财务 可行性 分析 产品 增产 减产 财务 可行性 分析 生产 设备 改造 更新 决策分析 投资 项目 现金流 分析 投资 项目 评价 方法 现值 法分析 资金 时间 价值 分析 综合 案例 演练 公司 费用 控制 公司 费用 控制 费用 方法 影响 费用 因素 分析 成本 中心 费用 控制 利润 中心 业绩考核 投资 中心 业绩 评价 利用 财务 数据分析 改善 绩效 公司财务 分析 核心 思路 关键 财务指标 解析 盈利 能力 分析 资产 回报率 股东权益 回报率 资产 流动 速率 风险 指数 分析 流动比率 负债 权益 比率 营运 偿债 能力 财务报表 综合 解读 综合 财务 信息 透视 公司 运作 水平 案例 分析 上市公司 财务状况 分析 评价 企业 运

In [39]:
from tqdm import tqdm

tqdm.pandas()  # 使用 tqdm 显示进度
# 将 apply 函数替换为 progress_apply 以使用 tqdm 显示处理进度
df['words'] = df[1].progress_apply(process)
df.head()







  0%|          | 0/10000 [00:00<?, ?it/s]





  0%|          | 12/10000 [00:00<01:48, 92.31it/s]





  0%|          | 30/10000 [00:00<01:32, 108.11it/s]





  0%|          | 39/10000 [00:00<01:41, 98.60it/s] 





  0%|          | 48/10000 [00:00<01:47, 92.89it/s]





  1%|          | 56/10000 [00:00<01:52, 88.60it/s]





  1%|          | 64/10000 [00:00<02:06, 78.28it/s]





  1%|          | 75/10000 [00:00<01:58, 83.73it/s]





  1%|          | 91/10000 [00:00<01:43, 95.95it/s]





  1%|          | 102/10000 [00:01<01:39, 99.77it/s]





  1%|          | 113/10000 [00:01<01:47, 92.30it/s]





  1%|          | 123/10000 [00:01<01:53, 87.08it/s]





  1%|▏         | 136/10000 [00:01<01:44, 94.54it/s]





  1%|▏         | 146/10000 [00:01<01:42, 96.12it/s]





  2%|▏         | 157/10000 [00:01<01:49, 90.08it/s]





  2%|▏         | 170/10000 [00:01<01:39, 99.22it/s]





  2%|▏         | 186/10000 [00:01<01:29, 109.68it/s]





  2%|▏         | 199/10000 [00:01<01:31,

 18%|█▊        | 1778/10000 [00:19<01:35, 86.10it/s]





 18%|█▊        | 1788/10000 [00:19<01:36, 85.25it/s]





 18%|█▊        | 1797/10000 [00:19<01:42, 79.72it/s]





 18%|█▊        | 1815/10000 [00:20<01:26, 94.21it/s]





 18%|█▊        | 1827/10000 [00:20<01:23, 98.23it/s]





 18%|█▊        | 1842/10000 [00:20<01:14, 109.58it/s]





 19%|█▊        | 1855/10000 [00:20<01:20, 101.52it/s]





 19%|█▊        | 1873/10000 [00:20<01:09, 116.80it/s]





 19%|█▉        | 1887/10000 [00:20<01:07, 119.76it/s]





 19%|█▉        | 1901/10000 [00:20<01:44, 77.42it/s] 





 19%|█▉        | 1912/10000 [00:21<01:44, 77.76it/s]





 19%|█▉        | 1922/10000 [00:21<02:06, 64.09it/s]





 19%|█▉        | 1939/10000 [00:21<01:43, 77.75it/s]





 20%|█▉        | 1950/10000 [00:21<01:38, 81.46it/s]





 20%|█▉        | 1961/10000 [00:21<01:33, 86.26it/s]





 20%|█▉        | 1977/10000 [00:21<01:20, 100.10it/s]





 20%|█▉        | 1989/10000 [00:22<01:50, 72.76it/s] 





 20%|██

 36%|███▌      | 3621/10000 [00:38<00:54, 116.65it/s]





 36%|███▋      | 3643/10000 [00:38<00:47, 133.32it/s]





 37%|███▋      | 3658/10000 [00:39<01:20, 79.05it/s] 





 37%|███▋      | 3674/10000 [00:39<01:09, 91.59it/s]





 37%|███▋      | 3691/10000 [00:39<01:01, 102.46it/s]





 37%|███▋      | 3708/10000 [00:39<00:55, 113.98it/s]





 37%|███▋      | 3722/10000 [00:39<00:53, 117.67it/s]





 37%|███▋      | 3736/10000 [00:39<00:57, 109.13it/s]





 37%|███▋      | 3749/10000 [00:39<00:58, 106.22it/s]





 38%|███▊      | 3762/10000 [00:40<00:56, 109.55it/s]





 38%|███▊      | 3778/10000 [00:40<00:52, 118.31it/s]





 38%|███▊      | 3796/10000 [00:40<00:48, 129.03it/s]





 38%|███▊      | 3811/10000 [00:40<00:48, 127.80it/s]





 38%|███▊      | 3825/10000 [00:40<00:53, 115.05it/s]





 38%|███▊      | 3838/10000 [00:40<00:53, 115.97it/s]





 39%|███▊      | 3851/10000 [00:40<00:55, 110.67it/s]





 39%|███▊      | 3863/10000 [00:40<00:55, 110.19it/s]





 55%|█████▌    | 5548/10000 [00:57<00:42, 105.00it/s]





 56%|█████▌    | 5560/10000 [00:58<00:53, 82.53it/s] 





 56%|█████▌    | 5572/10000 [00:58<00:54, 80.60it/s]





 56%|█████▌    | 5581/10000 [00:58<01:05, 67.03it/s]





 56%|█████▌    | 5592/10000 [00:58<01:01, 71.49it/s]





 56%|█████▌    | 5600/10000 [00:58<01:04, 68.18it/s]





 56%|█████▌    | 5610/10000 [00:58<00:58, 75.38it/s]





 56%|█████▌    | 5621/10000 [00:59<00:53, 81.39it/s]





 56%|█████▋    | 5634/10000 [00:59<00:47, 91.67it/s]





 56%|█████▋    | 5650/10000 [00:59<00:42, 103.11it/s]





 57%|█████▋    | 5662/10000 [00:59<00:52, 83.06it/s] 





 57%|█████▋    | 5672/10000 [00:59<00:55, 77.36it/s]





 57%|█████▋    | 5691/10000 [00:59<00:46, 92.71it/s]





 57%|█████▋    | 5708/10000 [00:59<00:39, 107.36it/s]





 57%|█████▋    | 5722/10000 [00:59<00:38, 109.99it/s]





 57%|█████▋    | 5737/10000 [01:00<00:36, 116.76it/s]





 57%|█████▊    | 5750/10000 [01:00<00:42, 100.83it/s]





 58%|█

 74%|███████▍  | 7398/10000 [01:16<00:27, 95.60it/s]





 74%|███████▍  | 7409/10000 [01:17<00:26, 96.88it/s]





 74%|███████▍  | 7425/10000 [01:17<00:23, 107.66it/s]





 74%|███████▍  | 7437/10000 [01:17<00:24, 105.24it/s]





 74%|███████▍  | 7449/10000 [01:17<00:28, 90.08it/s] 





 75%|███████▍  | 7460/10000 [01:17<00:27, 91.68it/s]





 75%|███████▍  | 7470/10000 [01:17<00:27, 92.46it/s]





 75%|███████▍  | 7480/10000 [01:17<00:31, 79.87it/s]





 75%|███████▍  | 7489/10000 [01:17<00:30, 82.66it/s]





 75%|███████▌  | 7506/10000 [01:18<00:25, 96.07it/s]





 75%|███████▌  | 7528/10000 [01:18<00:21, 113.81it/s]





 76%|███████▌  | 7553/10000 [01:18<00:18, 133.86it/s]





 76%|███████▌  | 7570/10000 [01:18<00:25, 96.65it/s] 





 76%|███████▌  | 7584/10000 [01:18<00:26, 91.86it/s]





 76%|███████▌  | 7596/10000 [01:18<00:26, 92.00it/s]





 76%|███████▌  | 7610/10000 [01:18<00:23, 100.34it/s]





 76%|███████▌  | 7622/10000 [01:19<00:23, 100.24it/s]





 76%|█

 93%|█████████▎| 9258/10000 [01:36<00:08, 84.12it/s]





 93%|█████████▎| 9268/10000 [01:36<00:08, 88.33it/s]





 93%|█████████▎| 9281/10000 [01:36<00:07, 95.57it/s]





 93%|█████████▎| 9298/10000 [01:36<00:06, 102.09it/s]





 93%|█████████▎| 9310/10000 [01:36<00:06, 106.88it/s]





 93%|█████████▎| 9326/10000 [01:36<00:05, 116.12it/s]





 93%|█████████▎| 9339/10000 [01:37<00:05, 113.67it/s]





 94%|█████████▎| 9358/10000 [01:37<00:05, 124.18it/s]





 94%|█████████▎| 9372/10000 [01:37<00:05, 118.72it/s]





 94%|█████████▍| 9387/10000 [01:37<00:04, 124.44it/s]





 94%|█████████▍| 9400/10000 [01:37<00:05, 104.48it/s]





 94%|█████████▍| 9412/10000 [01:37<00:05, 106.67it/s]





 94%|█████████▍| 9424/10000 [01:37<00:05, 104.58it/s]





 94%|█████████▍| 9441/10000 [01:37<00:04, 115.81it/s]





 95%|█████████▍| 9457/10000 [01:38<00:04, 123.35it/s]





 95%|█████████▍| 9470/10000 [01:38<00:06, 85.65it/s] 





 95%|█████████▍| 9481/10000 [01:38<00:05, 87.37it/s]





 

,0,1,words
37029,1,data/trec06c/data/123/129,"[恋爱, 第三次, 告诉, 再见面, 时间, 我要, 考研, 考到, 北京, 是否是, 喜欢..."
2257,0,data/trec06c/data/007/157,"[欣欣, 签约, 推出, 中国, 第一个, 彩铃, 歌手, 稀稀, 龙乐, 公司, 签约, ..."
50881,1,data/trec06c/data/169/181,"[男生, 思路, 简单, 心痛, 直说, 原因, 不让, 担心, 他累, 不去, 撒娇, 撒..."
10843,0,data/trec06c/data/036/043,[]
4689,0,data/trec06c/data/015/189,"[本港, 会计师, 权威机构, 香港, 瑞丰, 会计师, 事务所, 注册, 海外, 国际, ..."


把分词结果编码为可以输入算法的向量。这里用到自然语言领域常用的 Word2vec 方法。

In [40]:
from gensim.models import Word2Vec
from tqdm import tqdm_notebook

# 移除一些不必要的警告
import warnings
warnings.filterwarnings('ignore')

# 导入上面保存的分词数组
data = df['words']

# 训练 Word2Vec 浅层神经网络模型
w2v_model = Word2Vec(size=100, min_count=10)
w2v_model.build_vocab(data)
w2v_model.train(data, total_examples=w2v_model.corpus_count, epochs=5)
w2v_model

In [41]:
def sum_vec(text):
    # 对每个句子的进行词向量求和计算
    vec = np.zeros(100).reshape((1, 100))
    for word in text:
        try:
            # 得到句子中每个词的词向量并累加在一起
            vec += w2v_model[word].reshape((1, 100))
        except KeyError:
            continue
    return vec

In [42]:
# 将词向量保存为 Ndarray
data_vec = np.concatenate([sum_vec(z) for z in tqdm_notebook(data)])
data_vec.shape

(10000, 100)

### 数据划分及建模

In [50]:
from sklearn.model_selection import train_test_split

feature_data = data_vec
label_data = df[0].values
# 分割数据
X_train, X_test, y_train, y_test = train_test_split(
    feature_data, label_data, test_size=0.2, random_state=4)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8000, 100), (2000, 100), (8000,), (2000,))

由于 scikit-learn 中的多项式模型规定传入的矩阵必须非负，这里使用伯努利模型

In [51]:
from sklearn.naive_bayes import BernoulliNB

model = BernoulliNB()  # 定义伯努利模型分类器
model.fit(X_train, y_train)  # 模型训练
y_pred = model.predict(X_test)  # 模型预测
y_pred

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [52]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.9475